In [32]:
import requests
from pathlib import Path
import io
import zipfile
import os
import pandas as pd
import shutil
from pydantic import BaseModel
from typing import Dict, List, Optional, Any

In [14]:
flicker_dir = Path("../data/flicker")
raw_data_dir = Path("../data/raw_dataset")
flicker_dir.mkdir(parents=True, exist_ok=True)
raw_data_dir.mkdir(parents=True, exist_ok=True)

In [15]:
'''Get dataset from flicker'''
url = "https://www.lirmm.fr/~poncelet/Ressources/flickr_subset2.zip"
print("Requesting...\n")
response = requests.get(url)
if response.status_code == 200:
    print("Téléchargement réussi. Extraction...")
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        # Extraire sans ajouter de sous-dossier supplémentaire
        for member in zip_ref.namelist():
            # Corrige les chemins pour ignorer un éventuel prefixe flickr_subset2/
            member_path = member
            if member.startswith("flickr_subset2/"):
                member_path = member[len("flickr_subset2/"):]
            target_path = flicker_dir / member_path

            # Si c'est un répertoire, on le crée
            if member.endswith("/"):
                target_path.mkdir(exist_ok=True, parents=True)
            else:
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                with zip_ref.open(member) as source, open(target_path, "wb") as target:
                    target.write(source.read())
    print(f"Données extraites dans : {flicker_dir}")
else:
    print("Échec du téléchargement. Code HTTP :", response.status_code)


Requesting...

Téléchargement réussi. Extraction...
Données extraites dans : ../data/flicker


In [24]:
caption_csv_path = flicker_dir / "captions.csv"
df = pd.read_csv(caption_csv_path)
print(df.columns)
print(df.iloc[0])

Index(['image_path', 'label', 'caption'], dtype='object')
image_path                        images/dog/dog_001.jpg
label                                                dog
caption       A black dog and a spotted dog are fighting
Name: 0, dtype: object


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class ImageCLIPDataset(Dataset):
    def __init__(self, img, labels, flicker_dir):
        self.img_path = flicker_dir / img
        self.label = labels
    
    def __getitem__(self, idx):
        return self._get_img()[idx], self.y[idx]
    
    def _get_img(self):
        return Image.open(self.img_path).convert("RGB")
    

        


In [41]:
from sklearn.model_selection import train_test_split


X = df["img_path"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=11)

KeyError: 'img_path'

In [ ]:
train_dataset = ImageCLIPDataset(X_train, y_train)
test_dataset = ImageCLIPDataset(X_test, y_test)

